In [1]:
pip install tensorflow transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 5.2 MB/s eta 0:00:00
  Using cached absl_py-2.0.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached h5py-3.10.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (2.5 kB)
  Using cached libclang-16.0.6-py2.py3-none-macosx_11_0_arm64.whl.metadata (5.2 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached wrapt-1.14.1-cp310-cp310-macosx_11_0_arm64.whl (35 kB)
  Using cached grpcio-1.59.3-cp310-cp310-macosx_12_0_universal2.whl.metadata (4.0 kB)
  Using cached regex-2023.10.3-cp310-cp310-macosx_11_0_arm64.whl.metadata (40 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
  Using cached wheel-0.42.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached Markdown-3.5.1-py3-none-any.whl.metadata (7.1 kB)
  Using cac

In [1]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
import pandas as pd

/Users/matthewieraci/.pyenv/versions/3.10.6/envs/fake_news_detector/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('data/merged_data.csv')
df.sample(10)

,text,is_fake
9951,Wilbur Ross spoke with WaPo about true tax r...,1
22134,The partisan divide over the House Intelligenc...,0
20357,The Trump administration will soon begin a rev...,0
25583,"WHITE PLAINS, New York Democratic U.S. preside...",0
18026,The U.S. Department of Justice’s move to block...,0
4071,GOP presidential nominee Donald Trump has been...,1
34710,British transport minister Chris Grayling said...,0
15188,Whether federal spending legislation crosses t...,1
30084,Britain has made substantive changes to its pr...,0
1383,"Over the weekend, President Trump signed a mas...",1


In [3]:
df['is_fake'].value_counts(normalize=True)

is_fake
0    0.548192
1    0.451808
Name: proportion, dtype: float64

In [4]:
X = df['text']
y = df['is_fake']
train_texts, test_texts, train_labels, test_labels = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=42,
    shuffle=True
    )

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize(texts):
    return tokenizer(texts, padding=True, truncation=True, return_tensors='tf', max_length=400)

train_encodings = tokenize(train_texts.tolist())
test_encodings = tokenize(test_texts.tolist())


In [6]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
))

In [9]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [11]:
model.fit(train_dataset.shuffle(100).batch(32), epochs=3, batch_size=32)

Epoch 1/3
 10/846 [..............................] - ETA: 4:11:58 - loss: 0.9434 - accuracy: 0.4781

KeyboardInterrupt: 

In [ ]:
model.evaluate(test_dataset.batch(32))